In [1]:
%load_ext autoreload
%autoreload 2
import time
import tensorflow as tf
import p8_util

Using TensorFlow backend.


In [16]:
OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
datadir = './data'
is_tensorboard = True

In [17]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR_TB)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')

In [15]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

kill: (6120)kill: (6126)

In [11]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)

Using TensorFlow backend.


p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump


In [5]:
import p8_util_config
#-----------------------------------------------------------------
# Get the number of convolutional layers for CNN network
# This will fixe the equivalent parameter of AdaNet num layers.
#-----------------------------------------------------------------
num_layers = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']['conv_layer_num']
print("\n Num layers= {}".format(num_layers))
oNNAdaNetBuilder = p8_util._NNAdaNetBuilder(p8_util_config.dict_adanet_config, num_layers=num_layers)


 Num layers= 0

*** _NNAdaNetBuilder : NN Type=CNNBase


In [6]:
#help(tf.train.get_global_step)

In [7]:
# Parameters that will be provided to model_fn
import p8_util_config


params = {'net_builder':oNNAdaNetBuilder}
nn_type = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type']
print("\n NN Type= {}".format(nn_type))
my_config = p8_util.make_config(nn_type,output_dir=OUTPUT_DIR, is_restored=False)  
classifier = tf.estimator.Estimator(model_fn=p8_util.my_model_fn, params=params, config=my_config)
start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train, p8_util_config.NUM_EPOCHS\
                          , tuple_dimension=tuple_dimension\
                          , batch_size=p8_util_config.BATCH_SIZE),
        max_steps=p8_util_config.MAX_STEPS),
    
    eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("test", x_test, y_test, p8_util_config.NUM_EPOCHS\
                           , tuple_dimension=tuple_dimension\
                           , batch_size=p8_util_config.BATCH_SIZE),
        steps=None,
        throttle_secs=1))
end_time = time.time()


 NN Type= CNNBase
INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/CNNBase', '_tf_random_seed': 42, '_save_summary_steps': 5, '_save_checkpoints_steps': 5, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f923b2e89b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and eva

INFO:tensorflow:Saving dict for global step 50: eval_accuracy = 0.44680852, global_step = 50, loss = 1.0829052
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 50: ./tmp/baseline/CNNBase/model.ckpt-50
INFO:tensorflow:Loss for final step: 1.0800596.


In [8]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))



------------------------------------------------
Time (sec) 19.315252780914307
EVAL_ACCURACY: 0.44680851697921753
LOSS: 1.0829051733016968
GLOBAL_STEP: 50
